# Tag images from a file system
This notebook demonstrates how to use Splash-ML to scan a file system for files and then introduce them to splash-ml, how to save tag sets using the TagService in the tagging packages, and how to query on those tags.

The notebook uses mongomock to mimic a mongo database instance in memory. This could easily be replaced with a MongoClient from pymongo.

This notebook downloads and imports a pre-laballed data set that was generously provided by Carolin Sutter-Fella.

<cite>
    This data was acquired at beamline 12.3.2 at the Advanced Light
Source, which is a DOE Office of Science User Facility under
contract no. DE-AC02-05CH11231.
</cite>

The notebook downloads the dataset, which is a set of .tiff files and a csv that contains labels and the human tagger's confidence for each file. 
Lables include 'peaks', 'rings', 'rods' and 'arcs'. This notebook does the following:
- Download sample files locally into a folder called 'data/labelled'
- "Anonymize" the files, which copies them into a folder 'data/anonymous'
- Setups a TagService instance and introduce those files from 'data/anonymous' and their labels from the labels.csv file
- Demonstrates querying for those tags



In [1]:
import sys; sys.path.insert(0, '../..')
from datetime import datetime
import glob
import os
from pathlib import Path
import pandas as pd

from mongomock import MongoClient
from tagging.model import FileAsset, Tagger, TaggingEvent, AssetType, Tag, LABEL_NAME
from tagging.tag_service import TagService
from tagging.util.files import anonymize_copy

# By default, this does not require a mongo server running, but can use the mongomock
# library to create an in-memory simulation of mongo. This will be deleted when the 
# kernel is shutdown. 
in_memory_db = False
if in_memory_db:
    from mongomock import MongoClient
else:
    # for now, expects mongo running on localhost:27
    from pymongo import MongoClient

src_root_path = os.path.join("data", "labelled")
src_relative_path = "labelled"
dest_root = os.path.join("data", "anonymous", src_relative_path)



In [2]:
def download_zip():
    from tqdm import tqdm
    import time
    import requests
    from zipfile import ZipFile
    link = "https://portal.nersc.gov/cfs/als/splash_ml/labelled.zip"
    file_name = "data/labelled.zip"

    if os.path.exists('data'):
        print("Directory data/labelled exists...skipping")
        
    else:
        os.mkdir('data')
        with open(file_name, "wb") as file:
            print(f"downloading {link} to {file_name}")
            response = requests.get(link, stream=True)
            total_length = int(response.headers.get('content-length'))
            chunk_size = 4096  # 1 MB
            num_bars = int(total_length / chunk_size)
            print(total_length)
            if total_length is None:
                f.write(response.content)
            else:
                for chunk in tqdm(
                    response.iter_content(chunk_size=chunk_size), total=num_bars, unit='KB', desc="labelled.zip", leave=True, file=sys.stdout):
                    file.write(chunk)

    with ZipFile('data/labelled.zip', 'r') as zipObj:
        # Extract all the contents of zip file in different directory
        zipObj.extractall('data')
        print('File is unzipped into  "data/labelled" folder') 

download_zip()

Directory data/labelled exists...skipping
File is unzipped into  "data/labelled" folder


In [5]:
db = MongoClient()

#use glob to find all the files to ingest
paths = glob.glob('data/labelled/**/*.tif*', recursive=True)

# tag_svc instance to be used throughout creating and querying tags
tag_svc = TagService(db, db_name='tagging')

# tagger represents the entity creates tags on the assets...in this case, it's us!
tagger = tag_svc.create_tagger(Tagger(type="human", name="build_tag notebook"))

# event is recorded with each tag on each asset so we know when and what created the tags, in this case, it's us and now!
tagging_event = tag_svc.create_tagging_event(TaggingEvent(tagger_id=tagger.uid, run_time=datetime.now()))

# read the csv for tags
labels = pd.read_csv("data/labelled/labels.csv", header=[0])

num_files = 0
for src_root_path in paths:
    anonymous_file = anonymize_copy(src_root_path, src_relative_path, dest_root)
    # We'll take advantage of the fact that the anonymous file name is unique (hash of the file)
    anonymous_file_name = os.path.splitext(os.path.split(anonymous_file)[1])[0]
    # get label row from csv using file name
    file_name = os.path.splitext(os.path.split(src_root_path)[1])[0]
    
    3 # An Asset is a reference in the tagging database that stores information about something being tagged as well
    # as its tags. In this case, we have a file, so we're creating a FileAsset. Databroker assets will be available soon!
    # We pass the anonymous file's name, which is a hash of the file itself, as the uid. This is optional, a uid will be created
    # if none is passed.
    asset = FileAsset(uid=anonymous_file_name, uri=anonymous_file)
    # Associate this file with the name listed in the csv
    row = labels.loc[labels['image name'] == int(file_name)]
    tags = []
    tags.append(Tag(name=LABEL_NAME, value="peaks", confidence=row['peaks'].values[0], event_id=tagging_event.uid))
    tags.append(Tag(name=LABEL_NAME, value="rings", confidence=row['rings'].values[0], event_id=tagging_event.uid))
    tags.append(Tag(name=LABEL_NAME, value="rods", confidence=row['rods'].values[0], event_id=tagging_event.uid))
    tags.append(Tag(name=LABEL_NAME, value="arcs", confidence=row['arcs'].values[0], event_id=tagging_event.uid))
    asset.tags = tags
    tag_svc.create_asset(asset)
    num_files += 1
print(f"Anonymized, imported and tagged {num_files} files")

DuplicateKeyError: E11000 duplicate key error collection: tagging.tagger index: model_info.label_index_1 dup key: { model_info.label_index: null }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'model_info.label_index': 1}, 'keyValue': {'model_info.label_index': None}, 'errmsg': 'E11000 duplicate key error collection: tagging.tagger index: model_info.label_index_1 dup key: { model_info.label_index: null }'}

Now that we have loaded the tagging database, we can do some queries on what we have. First, find random tagging events.

In [6]:
assets_with_peaks = tag_svc.find_assets(**{"tags.value": "peaks"})
for asset in assets_with_peaks:
    tag = [tag for tag in asset.tags if tag.value == "peaks"]
    print(f"uid: {asset.uid} at {asset.uri} has peak tag: {tag}")

uid: 7d6928665d52239499f9501b87241fde at data/anonymous/labelled/6c196513838facc8a850bb15ed0e4b51/7d6928665d52239499f9501b87241fde.tif has peak tag: [Tag(name='label', value='peaks', confidence=0.1, event_id='68c0e5b2-7a8c-4282-9ff1-ac763a3e3fef')]
uid: cad59c5611adedcb793a09d071c780a0 at data/anonymous/labelled/6c196513838facc8a850bb15ed0e4b51/cad59c5611adedcb793a09d071c780a0.tif has peak tag: [Tag(name='label', value='peaks', confidence=0.1, event_id='68c0e5b2-7a8c-4282-9ff1-ac763a3e3fef')]
uid: cb202df1321627a31626511fe1b88b59 at data/anonymous/labelled/6c196513838facc8a850bb15ed0e4b51/cb202df1321627a31626511fe1b88b59.tif has peak tag: [Tag(name='label', value='peaks', confidence=0.5, event_id='68c0e5b2-7a8c-4282-9ff1-ac763a3e3fef')]
uid: 5de2fbd93962347f34bd868983ca081d at data/anonymous/labelled/6c196513838facc8a850bb15ed0e4b51/5de2fbd93962347f34bd868983ca081d.tif has peak tag: [Tag(name='label', value='peaks', confidence=0.2, event_id='68c0e5b2-7a8c-4282-9ff1-ac763a3e3fef')]
uid:

We query based on tags. (note that this signature will be enhanced to make confidence parameters a range and optional)